<a href="https://colab.research.google.com/github/goffycoder/paraphrase/blob/main/bart_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Installing the necessary libraries

In [1]:
!pip install transformers
!pip install datasets
!pip install rouge-score

# Step 2: Import the necessary libraries

In [2]:
import numpy as np
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
from rouge_score import rouge_scorer
import nltk

# Step 3: Load and preprocess the dataset

In [3]:
# Load the dataset
dataset = load_dataset("HHousen/ParaSCI")

  0%|          | 0/3 [00:00<?, ?it/s]

# Preprocess the dataset

In [4]:
def preprocess(example):
    return {'input_ids': tokenizer(example['sentence1'], truncation=True, padding='max_length', max_length=512)['input_ids'],
            'labels': tokenizer(example['sentence2'], truncation=True, padding='max_length', max_length=512)['input_ids']}


# Define the tokenizer

In [5]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Preprocess the training dataset


In [6]:
dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/338717 [00:00<?, ? examples/s]

Map:   0%|          | 0/6433 [00:00<?, ? examples/s]

Map:   0%|          | 0/4894 [00:00<?, ? examples/s]

# Split the dataset into training and validation sets

In [7]:
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

# Step 4: Define the compute metrics function

In [8]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


Define the compute_metrics function

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = scorer.score(decoded_labels, decoded_preds)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return result

# Step 5: Define the model, training arguments, and the trainer


In [10]:
# Define the model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')


Define the training arguments

In [11]:
!pip install accelerate -U


In [12]:
!pip install --upgrade accelerate

In [13]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

Define the trainer

In [14]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # validation dataset
    compute_metrics=compute_metrics,     # the compute_metrics function
)

# Step 6: Train and evaluate the model


In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
